In [ ]:
import numpy as np
import cv2

In [ ]:
lines_straight = np.genfromtxt('shelf0straightwithLinesPoints.csv', delimiter=',', usecols=(1,2))
lines_rrstraight = np.genfromtxt('shelfrrstraightwithLinesPoints.csv', delimiter=',', usecols=(1,2))

In [ ]:
print(lines_rrstraight.shape)
print(lines_straight.shape)
min(len(lines_rrstraight), len(lines_straight))

In [ ]:
def generate_matrix(points1, points2):
    point_pair_index = 0
    matrix = None
    while point_pair_index < min(len(lines_rrstraight), len(lines_straight)):
        matrix_row = generate_matrix_row12(points1[0], points1[1], points2[0], points2[1])
        matrix = matrix_row if matrix is None else np.append(matrix, matrix_row, axis=0)
        point_pair_index += 2
    return matrix


def generate_matrix_row12(start1, end1, start2, end2):
    """
    Equation for each row:
    matrix_row1 = [A*x_s1, A*y_s1, A, B*x_s1, B*y_s1, B, C*x_s1, C*y_s1, C]
    matrix_row2 = [A*x_e1, A*y_e1, A, B*x_e1, B*y_e1, B, C*x_e1, C*y_e1, C], where
    A = y_s2 - y_e2
    B = x_e2 - x_s2
    C = x_s2*y_e2 - x_e2*y_s2
    """
    constant_a = start2[1] - end2[1]
    constant_b = end2[0] - start2[0]
    constant_c = start2[0] * end2[1] - end2[0] * start2[1]
    matrix_row1 = np.array([constant_a * start1[0], constant_a * start1[1], constant_a,
                            constant_b * start1[0], constant_b * start1[1], constant_b,
                            constant_c * start1[0], constant_c * start1[1], constant_c])
    matrix_row2 = np.array([constant_a * end1[0],   constant_a * end1[1],   constant_a,
                            constant_b * end1[0],   constant_b * end1[1],   constant_b,
                            constant_c * end1[0],   constant_c * end1[1],   constant_c])
    return np.append(matrix_row1.reshape(1, len(matrix_row1)),
                     matrix_row2.reshape(1, len(matrix_row2)),
                     axis=0)


matrix_m = generate_matrix(lines_straight, lines_rrstraight)
print(matrix_m.shape)

In [ ]:
u, s, v = np.linalg.svd(matrix_m, full_matrices=False)

homography = v[np.argmin(s)].reshape(3, 3)
homography = homography / homography[2, 2]

print(homography)
np.savetxt('shelfhomo.csv', homography, delimiter=',')

In [ ]:
def euclidean_squared(x, y):
    """Squared euclidean distance between 2 1D arrays"""
    return np.sum(np.subtract(x, y) ** 2)


def calculate_transfer_error_symmetric(x, x_tick, homography):
    """
    Calculate symmetric transfer error between 2 similar points using a given homography
    d = sqrt(d(x, x'.inv(H))^2 + d(x', x.H)^2)

    :param x: (1, 2) array of coordinates of ONE original point
    :param x_tick: (1, 2) array of coordinates of ONE similar point
    :param homography: homography matrix of the transformation
    :return: symmetric transfer error
    """
    x_tick_transformed = np.dot(homography, np.append(x, 1.))
    x_transformed = np.dot(np.linalg.inv(homography), np.append(x_tick, 1.))
    squared_distance = euclidean_squared(x, x_transformed[:2]) + euclidean_squared(x_tick, x_tick_transformed[:2])

    return np.sqrt(squared_distance)

error = 0.0
for i in range(10):
    error += calculate_transfer_error_symmetric(lines_rrstraight[i, :2], lines_straight[i, :2], homography)
print(error)

A large number of matching points would allow recovery from bad matches. 

In [ ]:
def make_homog(point):
    if len(point) == 2:
        return np.append(point,[1])
    elif len(point) == 3:
        point = np.array(point,dtype=float)
        point = point/point[-1]
        return point
    else:
        print("Unknown size of point " , point)
        return -1

In [ ]:
b1 = make_homog(lines_straight[10])
t1 = make_homog(lines_straight[11])
b2 = make_homog(lines_straight[4])
t2 = make_homog(lines_straight[6])
horizon1 = make_homog(lines_straight[12])
horizon2 = make_homog(lines_straight[13])

print('\n'.join([str(b1),
                 str(t1),
                 str(b2),
                 str(t2),
                 str(horizon1),
                 str(horizon2)]))

In [ ]:
# Find horizontal line through horizon1 and horizon2
horizon = np.cross(horizon2, horizon1)
horizon_ = make_homog(horizon)
print("horizon : " , horizon_)

# Find b1b2
b1b2 = np.cross(b1, b2)
b1b2_ = make_homog(b1b2)
print("Line through b1, b2: ", b1b2_)

# Intersection u between b1b2 and horizon (vanishing point):
u = make_homog(np.cross(b1b2_, horizon_))
print("Intersection u: ", u)

# Find v intersection of b2t2, b1t1
b2t2 = make_homog(np.cross(b2, t2))
b1t1 = make_homog(np.cross(b1, t1))
v = make_homog(np.cross(b2t2, b1t1))
print("Intersection v of b2t2, b1t1: ", v)

# Find transfered point t1_tilda, intersection of t1u and vb2
t1u = make_homog(np.cross(t1, u))
vb2 = make_homog(np.cross(v, b2))
t1_tilda = make_homog(np.cross(t1u, vb2))
print("Transfered point t1_tilda: ", t1_tilda)



In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

img_data = cv2.imread('shelf0straightwithLines.jpg')
plt.figure(figsize=(15,15))
plt.imshow(img_data)
plt.scatter(t1_tilda[0],t1_tilda[1])
plt.show()

In [ ]:
from scipy.spatial.distance import euclidean

tilda_t1_dist = [euclidean(t1_tilda, b2), 1]
t2_dist = [euclidean(t2, b2), 1]
v_dist = [euclidean(v, b2), 1]

print ("V : ", v_dist)

H = [[1, v_dist[0]], [0,1]]
print("H : " , H)

T2 = np.dot(H, v_dist)
print( "T2 : " , T2)

# Ratio of distance
d1_d2 = (tilda_t1_dist[0]*(v_dist[0] - t2_dist[0]))/(t2_dist[0]*(v_dist[0] - tilda_t1_dist[0]))
print("Ratio of distance d1/d2 = " , d1_d2)

calculated_dist = 125/d1_d2

print("Length of 5-7 in cm : " , calculated_dist , "cm")

In [ ]:
print("Error in cm: ", (32.7 - calculated_dist))